<a href="https://colab.research.google.com/github/rock42069/Beyond_The_Buzz/blob/main/ML_MODEL_for_beyond_the_buzz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BEYOND THE BUZZ**

> CONTRIBUTER: Trijal Srivastava

 
 



Hi and welcome to my submission!
I will try my best to keep an engaging commentary running with the code. The code represents my final solution, but with my text blocks I will try to give you some insight into how we reached this.

Let's begin by importing necessary libraries.





In [1]:
#for handling data and ploting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#for making neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

The next two blocks help extract data from the .csv files provided, and then use it for training the model, converting them into numpy arrays.


In [2]:
train = pd.read_csv('/content/trainrprocessed.csv')

In [3]:
numerical_cols = ['PARAMETER_1', 'PARAMETER_2', 'PARAMETER_3', 'PARAMETER_4', 'PARAMETER_5', 'PARAMETER_6', 'PARAMETER_7', 'PARAMETER_8', 'PARAMETER_9']
scaler = StandardScaler()

train[numerical_cols] = scaler.fit_transform(train[numerical_cols])

Here I have checked the correlation between the parameters, as it is clear from the obtained matrix that there is no correlation between any of the variables

In [4]:
corr_matrix = train.corr()
#corr_matrix

I have stored all parameters in a numpy array named data and all the verdicts in a numpy array named y_array

In [5]:
y_array = train['VERDICT'].values
 
data = train[['PARAMETER_1', 'PARAMETER_2','PARAMETER_3','PARAMETER_4','PARAMETER_5','PARAMETER_6','PARAMETER_7','PARAMETER_8','PARAMETER_9']].values
 


Now I have modeled the neural nets. They are powerful universal function approximators, and amongst all the models we used this one, gave the best results.

In [6]:
model = Sequential(
    [               
        tf.keras.Input(shape=(9,)),    
        Dense(128,activation ='relu'),
        Dense(64,activation ='relu'),
        Dense(32,activation ='relu'),
        Dense(1,activation ='sigmoid')
        
        
      
    ], name = "my_model" 
)

Found the best architecture for our Neural Network by trial-and-error (not shown here). I observed that having 3 layers improved both the training and validation loss as compared to 2 layers, and adding an extra 4th layer worsened the validation loss due to overfitting of data. The number of neurons in each layer was determined by trying several variations. On making 128 neurons in one of the layers, we were able to minimize loss satisfactorily. 

I tried several learning rates and settled on 0.001 because decreasing it further was significantly increasing the training time without much improvement on the results. 

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
n_splits = 5
test_size = 0.2
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)
for train_index, test_index in sss.split(data, y_array):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y_array[train_index], y_array[test_index]

In [8]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics = ['accuracy']

)

history= model.fit(
    X_train,y_train,
    batch_size=90,
    epochs=30
)

Epoch 1/30
262/262 [==============================] - 2s 2ms/step - loss: 0.2665 - accuracy: 0.9302
Epoch 2/30
262/262 [==============================] - 1s 2ms/step - loss: 0.2245 - accuracy: 0.9416
Epoch 3/30
262/262 [==============================] - 1s 2ms/step - loss: 0.2201 - accuracy: 0.9416
Epoch 4/30
262/262 [==============================] - 1s 3ms/step - loss: 0.2177 - accuracy: 0.9415
Epoch 5/30
262/262 [==============================] - 1s 4ms/step - loss: 0.2151 - accuracy: 0.9415
Epoch 6/30
262/262 [==============================] - 1s 4ms/step - loss: 0.2135 - accuracy: 0.9416
Epoch 7/30
262/262 [==============================] - 2s 7ms/step - loss: 0.2116 - accuracy: 0.9417
Epoch 8/30
262/262 [==============================] - 2s 7ms/step - loss: 0.2110 - accuracy: 0.9416
Epoch 9/30
262/262 [==============================] - 1s 5ms/step - loss: 0.2094 - accuracy: 0.9420
Epoch 10/30
262/262 [==============================] - 1s 4ms/step - loss: 0.2088 - accuracy: 0.9423

In [9]:
prediction = model.predict(X_test)
prediction = np.where(prediction >= 0.5, 1, 0)

loss, accuracy = model.evaluate(X_test, y_test)

# Print the evaluation results
print('Test loss:', loss)
print('Test accuracy:', accuracy)


185/185 [==============================] - 0s 2ms/step - loss: 0.2079 - accuracy: 0.9399
Test loss: 0.20791491866111755
Test accuracy: 0.939949095249176


# PERFORMANCE
Here we have tested our data using various performance metrics just for personal reference.


In [10]:
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, prediction)
from sklearn.metrics import f1_score
f1 = f1_score(y_test, prediction)

 

print("AUC-ROC: ", auc_roc)
print("F1 Score: ", f1)

AUC-ROC:  0.5086431629750349
F1 Score:  0.9690071791279987


In [13]:
test = pd.read_csv('/content/test.csv')
data_test = test[['PARAMETER_1', 'PARAMETER_2','PARAMETER_3','PARAMETER_4','PARAMETER_5','PARAMETER_6','PARAMETER_7','PARAMETER_8','PARAMETER_9']].values
prediction = model.predict(data_test)
prediction = np.where(prediction >= 0.5, 1, 0)


s0 = 0
s1 = 0

for i in range(len(prediction)):
    if prediction[i] == 0:
        s0 += 1
    else:
        s1 +=1
print(s0,s1)
prediction = prediction.reshape(-1)
df = pd.DataFrame({'Prediction': prediction})
df.to_csv('predictions.csv', index=False)



1842/1842 [==============================] - 3s 1ms/step
2405 56516
